<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Neural networks </h2>	


Build a neural network classifier using an architecture of your choosing. This application
of deep learning can be done in PyTorch, TensorFlow, or a framework of your choice. This is the
industry standard for image classification. Describe your network and assess its performance. To
receive extra credit, your neural network classifier must outperform your other methods.



In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from datainput import *
from preprocess import *

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_path = './20_categories_training/'
# val_path = './20_Validation/'
# try not to printout train_data['Pictures'] directly, takes a while
train_data = read_train_data(train_path)
# test_data = read_test_data(val_path)

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Reshape
from keras.regularizers import l1
from keras.models import Model

def NN(inputshape = (300,400,3), l = 1e-6):
    
    regularizer_strength = l
    input_img = Input(shape=inputshape)  # y, x
    x = Conv2D(32, (3, 3), activation='relu', 
            padding='same', activity_regularizer=l1(regularizer_strength))(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', 
            padding='same', activity_regularizer=l1(regularizer_strength))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', 
            padding='same', activity_regularizer=l1(regularizer_strength))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', 
            padding='same', activity_regularizer=l1(regularizer_strength))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', 
            padding='same', activity_regularizer=l1(regularizer_strength))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(20, activation='softmax')(x)

    model = Model(input_img, output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = NN()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 300, 400, 3)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 300, 400, 32)      896       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 150, 200, 32)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 150, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 75, 100, 32)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 75, 100, 32)       9248      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 38, 50, 32)        0   

In [19]:
def train_test_split(df):
    """Split data into training and testing(validation)"""
    n, d = df.shape
    shuffled = np.arange(n)
    np.random.shuffle(shuffled)
    train_n = int(n*0.8)
    test_n = n-train_n
    train_idx = shuffled[:train_n]
    test_idx = shuffled[train_n:]
    return df.iloc[train_idx,:d-1], df.iloc[train_idx,d-1], df.iloc[test_idx,:d-1], df.iloc[test_idx,d-1]
    
def accuracy(pred, actual):
    return np.sum(pred==actual)/len(pred)

def add_dimension(image):
    if len(image.shape) == 2:
        return np.stack([image,image,image], axis = -1)
    return image

def cnn_feature_frame(df):
    # input original training_data set
    FE = feature_extract()
    images = df.Pictures
    # trim all images
    print('Trim all images..')
    images = FE.trim_all(images)
    df_X = pd.DataFrame()
    # convert all images to same size first 400x300
    images = pd.Series(images.apply(lambda x: FE.resize_image(x, 400, 300)))
    df_X['Pictures'] = images.apply(add_dimension)
    
    if 'Encoding' in df.columns:
        df_X['Encoding'] = df.Encoding
    return df_X



In [23]:
outliers = train_data[train_data.Pictures.apply(lambda x: len(x.shape)==2)]

In [27]:
X_train, Y_train, X_test, Y_test = train_test_split(cnn_feature_frame(train_data))


Trim all images..
